In [2]:
!pip install langgraph

  Using cached langgraph-0.4.8-py3-none-any.whl (152 kB)
  Using cached langgraph_checkpoint-2.0.26-py3-none-any.whl (44 kB)
  Using cached langgraph_prebuilt-0.2.2-py3-none-any.whl (23 kB)
  Using cached langgraph_sdk-0.1.70-py3-none-any.whl (49 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 446.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 1.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os, glob
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from pypdf import PdfReader
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

In [13]:
docs = os.listdir('docs/')


In [14]:
def extract_text_pypdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [15]:
contents = []

documents = [Document]

for x in range(0, len(docs)):
    
    contents.append(extract_text_pypdf(f'docs/{docs[x]}'))
    documents.append(Document(page_content=contents[x], metadata = {'source': f'docs/{docs[x]}'}))
    # # Documents.append({'content': f'{contents[x]}', 'metadata': {"source": f"docs/{docs[x]}"}})

documents[1:]


[Document(metadata={'source': 'docs/probability1.pdf'}, page_content='Week-2\nLecture-2\n2.2 Sample Spaces and Algebra Sets\nThe starting point for studying probability is the definition of four key terms:\nexperiment, sample outcome, sample space, and event.\nBy an experiment we will mean any procedure that can be repeated, theo-\nretically, an infinite number of times and has a well-defined set of possible out-\ncomes.\nEach of the potential eventualities of an experiment is referred to as asample\noutcome, s, and their totality is called the sample space, S. To signify the mem-\nbership of s in S, we write s ∈ S. Any designated collection of sample outcomes,\nincluding individual outcomes, the entire sample space, and the null set, consti-\ntutes an event. The latter is said to occur if the outcome of the experiment is\none of the members of the event.\nExample 2.2.1\nConsider the experiment of flipping a coin three times. What is the sample\nspace? Which sample outcomes make up the

In [16]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
vector_store = FAISS.from_documents(documents[1:], embeddings)

In [17]:
retriever = vector_store.as_retriever()


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

custom_prompt = """You are an calculus specialist who has been working tirelessly for 10 years. You really love your job
and answering to the questions by explaining the concept based on the provided document. If the information is not given in
the document, do not go further from that and just answer that you don't have enough information. 

Your task: Answer to the question in precise and understandable way based on the provided documents. 

Question: {question}
"""

real_prompt = PromptTemplate(
    template=custom_prompt
    input_variables=['question']
)

llm = ChatOpenAI(model='gpt-4o', openai_api_key=os.getenv('OPENAI_API_KEY'))

qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever, prompt=real_prompt)

NameError: name 'PromptTemplate' is not defined

In [19]:
# Q/A assisstant

while True:
    query = input("Ask your question: ")
    
    if query.lower() in ["exit", "quit"]:
        break
    response = qa_chain.run(query)
    print("AI Agent:", response)

In [20]:
qgen = [SystemMessage(content='You are a proffesional calculus tutor'), HumanMessage(content='Can you please generate me 10 questions for test preperation based on the avalable pdfs?')]

output = qa_chain.run(qgen)

/tmp/ipykernel_17361/540622423.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = qa_chain.run(qgen)


TypeError: expected string or buffer